In [1]:
import socket
import threading
import json
import time
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
import dlib
import face_recognition
from deepface import DeepFace
from dollarpy import Recognizer, Template, Point
import mediapipe as mp

In [3]:
connections = {}
server_sockets = []

In [4]:
def handle_client(conn, addr, client_name):
    """
    Handles communication with a connected client.
    """
    print(f"{client_name} connected from {addr}")
    connections[client_name] = conn
    try:
        while True:
            data = conn.recv(1024)  # Receive data
            if not data:
                break

            message = data[2:].decode('utf-8')

            message_dict = json.loads(message)

            if client_name == "Java":
                modelIndex = message_dict.get("ModelIndex", None)

                if modelIndex:
                    send_structured_message("Unity", "ChangeModel", modelIndex)

            print(f"Received from {client_name}: {message_dict}")
    except ConnectionResetError:
        print(f"{client_name} disconnected.")
    finally:
        conn.close()
        if client_name in connections:
            del connections[client_name]

In [5]:
def start_server(port, client_name):
    """
    Starts a server socket listening on a specified port for a specific client.
    """
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('127.0.0.1', port))
    server_socket.listen(5)
    server_sockets.append(server_socket)  # Track the server socket
    print(f"Server listening for {client_name} on port {port}...")
    try:
        while True:
            conn, addr = server_socket.accept()
            threading.Thread(target=handle_client, args=(conn, addr, client_name)).start()
    except Exception as e:
        print(f"Server for {client_name} on port {port} encountered an error: {e}")
    finally:
        server_socket.close()

In [6]:
def send_message_to(client_name, message):
    """
    Sends a message to a connected client.
    """
    if client_name in connections:
        try:
            encodedMessage = message.encode()
            connections[client_name].send(encodedMessage)
            print(f"Sending message: {encodedMessage}")
        except BrokenPipeError:
            print(f"Failed to send message to {client_name}. Connection might be closed.")
    else:
        print(f"{client_name} is not connected. {message} can't be sent.")

In [7]:
def send_structured_message(client_name, action, value):
    obj = {"Action": action, "Value": value}
    jsonData = json.dumps(obj)
    send_message_to(client_name, jsonData + "\n")

In [8]:
def close_all_connections():
    """
    Closes all client connections and server sockets.
    """
    print("Closing all connections...")
    for client_name, conn in list(connections.items()):
        try:
            conn.close()
            print(f"Closed connection for {client_name}")
        except Exception as e:
            print(f"Error closing connection for {client_name}: {e}")
        finally:
            del connections[client_name]

    for server_socket in server_sockets:
        try:
            server_socket.close()
            print("Server socket closed.")
        except Exception as e:
            print(f"Error closing server socket: {e}")

In [9]:
lower_red = np.array([0, 0, 0])
upper_red = np.array([0, 95, 10])

def detect_led(frame, lower_red, upper_red):
    """
    Detects an LED in the given frame and returns its bounding rectangle
    along with the modified frame with a rectangle drawn around the detected LED, if any.
    """
    # Convert the frame to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a mask for the LED light color using the threshold values
    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Apply morphological operations to remove noise
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check if any contours were found
    if len(contours) > 0:
        # Find the largest contour
        largest_contour = max(contours, key=cv2.contourArea)

        # Get the bounding rectangle for the contour
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Draw a rectangle around the LED light
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Return the position of the LED as the center of the bounding box
        return frame, (x + w // 2, y + h // 2)

    return frame, None

In [10]:
ali_image = face_recognition.load_image_file("people/ali.jpg")
ali_face_encoding = face_recognition.face_encodings(ali_image)[0]

kenzy_image = face_recognition.load_image_file("people/kenzy.jpg")
kenzy_face_encoding = face_recognition.face_encodings(kenzy_image)[0]

known_face_encodings = [
    ali_face_encoding,
    kenzy_face_encoding
]
known_face_names = [
    "Ali ElRogbany",
    "Kenzy"
]

# Initialize lists for storing detected face information
face_locations = []
face_encodings = []
face_names = []

# Facial Recognition Function
def detect_faces(frame, known_face_encodings, known_face_names):
    rgb_small_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 

    face_locations = []
    face_names = []

    # Detect faces and their encodings
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        # Find the best match for the detected face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        face_names.append(name)

    return face_locations, face_names, len(face_encodings) > 0


In [11]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")
gaze_data = []

def track_gaze(frame):
    colored_frame = frame.copy()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        shape = predictor(gray, face)

        landmarks = np.array([[p.x, p.y] for p in shape.parts()])

        left_eye_indices = [36, 37, 38, 39, 40, 41]
        right_eye_indices = [42, 43, 44, 45, 46, 47]

        left_eye = landmarks[left_eye_indices]
        right_eye = landmarks[right_eye_indices]

        left_eye_center = np.mean(left_eye, axis=0).astype(int)
        right_eye_center = np.mean(right_eye, axis=0).astype(int)

        avg_x = int((left_eye_center[0] + right_eye_center[0]) / 2)
        avg_y = int((left_eye_center[1] + right_eye_center[1]) / 2)

        gaze_data.append((avg_x, avg_y))

        cv2.circle(colored_frame, tuple(left_eye_center), 3, (0, 255, 0), -1)
        cv2.circle(colored_frame, tuple(right_eye_center), 3, (0, 255, 0), -1)
        cv2.circle(colored_frame, (avg_x, avg_y), 3, (0, 0, 255), -1)

    return colored_frame

In [12]:
def save_gaze_data_to_csv():
    """Transform gaze data list to pandas dataframe and save as CSV with a timestamped filename."""
    if not gaze_data:
        print("No gaze data to save.")
        return

    # Generate a timestamped filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"gaze_data_{timestamp}.csv"

    # Save data to CSV
    df = pd.DataFrame(gaze_data, columns=['x', 'y'])
    df.to_csv(f"gazedata/{file_name}", index=False)
    print(f"Gaze data saved to {file_name}")

    return df

In [13]:
def generate_heatmap(df, frame_size, output_path):
    """
    Generate a heatmap for gaze points from a DataFrame and save it as a PNG image.
    
    Parameters:
    - df: DataFrame containing 'x' and 'y' columns for gaze points.
    - frame_size: Tuple (width, height) of the frame.
    - output_path: Path to save the heatmap image (default is 'heatmap.png').
    """
    # Initialize heatmap array
    heatmap = np.zeros((frame_size[1], frame_size[0]))  # (height, width)

    # Accumulate gaze points
    for _, row in df.iterrows():
        x, y = int(row['x']), int(row['y'])
        if 0 <= y < frame_size[1] and 0 <= x < frame_size[0]:
            heatmap[y, x] += 1

    # Apply Gaussian blur
    heatmap = cv2.GaussianBlur(heatmap, (151, 151), 0)

    # Normalize for visualization
    if heatmap.max() > 0:
        heatmap /= heatmap.max()

    # Plot heatmap
    plt.imshow(heatmap, cmap='jet', alpha=0.8, extent=[0, frame_size[0], frame_size[1], 0])
    plt.colorbar(label="Gaze Intensity")
    plt.title("Gaze Heatmap")
    plt.xlabel("X-axis (pixels)")
    plt.ylabel("Y-axis (pixels)")

    # Save the plot as a PNG image
    plt.savefig(output_path, format='png')


In [14]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

right = Template('right', [
Point(256, 811, 1),
Point(18, 822, 1),
Point(244, 767, 1),
Point(15, 791, 1),
Point(233, 719, 1),
Point(12, 774, 1),
Point(222, 666, 1),
Point(22, 744, 1),
Point(222, 626, 1),
Point(31, 721, 1),
Point(220, 591, 1),
Point(32, 698, 1),
Point(215, 562, 1),
Point(28, 677, 1),
Point(211, 539, 1),
Point(27, 660, 1),
Point(213, 524, 1),
Point(31, 640, 1),
Point(220, 509, 1),
Point(35, 629, 1),
Point(227, 497, 1),
Point(44, 620, 1),
Point(236, 489, 1),
Point(54, 612, 1),
Point(250, 482, 1),
Point(64, 606, 1),
Point(263, 478, 1),
Point(76, 602, 1),
Point(280, 476, 1),
Point(90, 596, 1),
Point(299, 473, 1),
Point(109, 587, 1),
Point(316, 472, 1),
Point(133, 585, 1),
Point(331, 469, 1),
Point(155, 578, 1),
Point(345, 466, 1),
Point(178, 571, 1),
Point(359, 466, 1),
Point(197, 566, 1),
Point(372, 468, 1),
Point(211, 562, 1),
Point(378, 470, 1),
Point(221, 560, 1),
Point(383, 472, 1),
Point(225, 560, 1),
Point(383, 474, 1),
Point(224, 560, 1),
Point(378, 476, 1),
Point(214, 562, 1),
Point(371, 477, 1),
Point(202, 566, 1),
Point(358, 478, 1),
Point(187, 568, 1),
Point(347, 478, 1),
Point(170, 566, 1),
Point(334, 476, 1),
Point(152, 567, 1),
Point(325, 475, 1),
Point(135, 566, 1),
Point(316, 472, 1),
Point(121, 567, 1),
Point(308, 470, 1),
Point(116, 567, 1),
Point(309, 472, 1),
Point(117, 566, 1),
Point(319, 474, 1),
Point(122, 566, 1),
Point(329, 475, 1),
Point(135, 565, 1),
Point(340, 475, 1),
Point(155, 566, 1),
Point(356, 475, 1),
Point(180, 560, 1),
Point(375, 474, 1),
Point(203, 557, 1),
Point(390, 475, 1),
Point(228, 554, 1),
Point(404, 477, 1),
Point(245, 551, 1),
Point(416, 476, 1),
Point(267, 549, 1),
Point(424, 472, 1),
Point(285, 549, 1),
Point(429, 471, 1),
Point(293, 547, 1),
Point(431, 469, 1),
Point(297, 548, 1),
Point(431, 469, 1),
Point(298, 549, 1),
Point(431, 470, 1),
Point(297, 551, 1),
Point(430, 471, 1),
Point(294, 553, 1),
Point(430, 474, 1),
Point(290, 556, 1),
Point(428, 478, 1),
Point(281, 560, 1),
Point(428, 483, 1),
Point(271, 562, 1),
Point(424, 487, 1),
Point(259, 567, 1),
Point(418, 492, 1),
Point(244, 575, 1),
Point(406, 497, 1),
Point(229, 581, 1),
Point(393, 498, 1),
Point(211, 587, 1),
Point(375, 501, 1),
Point(189, 595, 1),
Point(356, 505, 1),
Point(168, 601, 1),
Point(337, 506, 1),
Point(146, 608, 1),
Point(317, 511, 1),
Point(124, 620, 1),
Point(298, 524, 1),
Point(105, 638, 1),
Point(154, 308, 1),
Point(0, 394, 1),
Point(162, 313, 1),
Point(3, 400, 1),
Point(175, 313, 1),
Point(10, 407, 1),
Point(199, 311, 1),
Point(20, 409, 1),
Point(223, 313, 1),
Point(41, 413, 1),
Point(270, 307, 1),
Point(59, 417, 1),
Point(319, 301, 1),
Point(76, 414, 1),
Point(343, 302, 1),
Point(86, 416, 1),
Point(359, 306, 1),
Point(96, 414, 1),
Point(366, 311, 1),
Point(107, 416, 1),
Point(377, 314, 1),
Point(120, 414, 1),
Point(386, 318, 1),
Point(126, 417, 1),
Point(388, 322, 1),
Point(133, 415, 1),
Point(388, 322, 1),
Point(137, 417, 1),
Point(387, 324, 1),
Point(140, 416, 1),
Point(385, 325, 1),
Point(143, 415, 1),
Point(383, 328, 1),
Point(143, 416, 1),
Point(378, 328, 1),
Point(140, 417, 1),
Point(373, 328, 1),
Point(133, 418, 1),
Point(367, 329, 1),
Point(125, 419, 1),
Point(357, 330, 1),
Point(112, 420, 1),
Point(347, 330, 1),
Point(101, 421, 1),
Point(338, 330, 1),
Point(93, 423, 1),
Point(329, 332, 1),
Point(82, 424, 1),
Point(319, 333, 1),
Point(72, 427, 1),
Point(307, 331, 1),
Point(60, 431, 1),
Point(296, 331, 1),
Point(48, 431, 1),
Point(288, 331, 1),
Point(38, 431, 1),
Point(278, 331, 1),
Point(29, 430, 1),
Point(268, 332, 1),
Point(23, 427, 1),
Point(262, 335, 1),
Point(19, 426, 1),
Point(257, 336, 1),
Point(13, 425, 1),
Point(251, 338, 1),
Point(10, 423, 1),
Point(247, 338, 1),
Point(6, 422, 1),
Point(244, 339, 1),
Point(3, 419, 1),
Point(243, 337, 1),
Point(1, 417, 1),
Point(242, 335, 1),
Point(0, 414, 1),
Point(244, 333, 1),
Point(1, 413, 1),
Point(247, 330, 1),
Point(1, 409, 1),
Point(250, 326, 1),
Point(2, 409, 1),
Point(254, 323, 1),
Point(6, 407, 1),
Point(260, 321, 1),
Point(11, 403, 1),
Point(265, 318, 1),
Point(15, 403, 1),
Point(274, 316, 1),
Point(23, 404, 1),
Point(283, 313, 1),
Point(34, 404, 1),
Point(297, 308, 1),
Point(47, 402, 1),
Point(309, 307, 1),
Point(59, 399, 1),
Point(320, 306, 1),
Point(68, 398, 1),
Point(332, 305, 1),
Point(81, 397, 1),
Point(345, 305, 1),
Point(99, 391, 1),
Point(352, 305, 1),
Point(110, 390, 1),
Point(362, 305, 1),
Point(121, 389, 1),
Point(367, 305, 1),
Point(129, 387, 1),
Point(372, 306, 1),
Point(135, 386, 1),
Point(373, 305, 1),
Point(141, 385, 1),
Point(373, 303, 1),
Point(145, 383, 1),
Point(370, 302, 1),
Point(143, 383, 1),
Point(368, 301, 1),
Point(139, 382, 1),
Point(364, 300, 1),
Point(135, 383, 1),
Point(356, 300, 1),
Point(127, 385, 1),
Point(344, 297, 1),
Point(119, 386, 1),
Point(342, 303, 1),
Point(111, 386, 1),
Point(334, 309, 1),
Point(104, 387, 1),
Point(326, 310, 1),
Point(93, 390, 1),
Point(317, 314, 1),
Point(83, 393, 1),
Point(309, 319, 1),
Point(70, 397, 1),
Point(297, 322, 1),
Point(56, 400, 1),
Point(289, 324, 1),
Point(47, 402, 1),
Point(282, 323, 1),
Point(35, 399, 1),
Point(272, 324, 1),
Point(26, 400, 1),
Point(264, 323, 1),
Point(23, 397, 1),
Point(258, 322, 1),
Point(17, 395, 1),
Point(251, 321, 1),
Point(12, 394, 1),
Point(249, 320, 1),
Point(8, 393, 1),
Point(247, 317, 1),
Point(2, 391, 1),
Point(243, 316, 1),
Point(0, 390, 1),
Point(243, 314, 1),
Point(-1, 390, 1),
Point(244, 312, 1),
Point(0, 389, 1),
Point(248, 310, 1),
Point(1, 388, 1),
Point(254, 309, 1),
Point(6, 388, 1),
Point(259, 309, 1),
Point(10, 386, 1),
Point(263, 309, 1),
Point(16, 386, 1),
Point(269, 307, 1),
Point(26, 386, 1),
Point(277, 307, 1),
Point(39, 387, 1),
Point(288, 307, 1),
Point(40, 386, 1),
Point(300, 303, 1),
Point(49, 386, 1),
Point(307, 303, 1),
Point(60, 386, 1),
Point(315, 301, 1),
Point(69, 386, 1),
Point(328, 297, 1),
Point(89, 383, 1),
Point(338, 296, 1),
Point(103, 380, 1),
Point(343, 291, 1),
Point(115, 378, 1),
Point(353, 292, 1),
Point(126, 381, 1),
Point(358, 294, 1),
Point(136, 377, 1),
Point(364, 293, 1),
Point(149, 379, 1),
Point(367, 294, 1),
Point(157, 378, 1),
Point(371, 298, 1),
Point(161, 373, 1),
Point(374, 299, 1),
Point(163, 377, 1),
Point(374, 303, 1),
Point(166, 378, 1),
Point(377, 305, 1),
Point(163, 382, 1),
Point(377, 309, 1),
Point(161, 384, 1),
Point(378, 313, 1),
Point(159, 386, 1),
Point(381, 319, 1),
Point(149, 393, 1),
Point(382, 328, 1),
Point(140, 396, 1),
Point(385, 346, 1),
Point(134, 399, 1),
Point(390, 366, 1),
Point(124, 401, 1),
Point(399, 385, 1),
Point(117, 411, 1),
Point(404, 403, 1),
Point(105, 418, 1),
Point(408, 426, 1),
Point(92, 431, 1),
Point(410, 457, 1),
Point(85, 439, 1),
Point(408, 487, 1),
Point(72, 453, 1),
Point(265, 486, 1),
Point(57, 470, 1),
Point(295, 527, 1),
Point(43, 484, 1),
Point(341, 576, 1),
Point(28, 500, 1),
Point(358, 630, 1),
Point(9, 521, 1),
Point(359, 689, 1),
Point(8, 533, 1),
Point(308, 658, 1),
Point(-6, 552, 1),
Point(298, 760, 1),
Point(2, 552, 1),
Point(303, 810, 1),
Point(20, 598, 1),
Point(296, 825, 1),
Point(25, 588, 1),
Point(234, 793, 1),
Point(17, 596, 1),
Point(217, 798, 1),
Point(22, 600, 1),
Point(252, 877, 1),
Point(21, 609, 1),
Point(249, 889, 1),
Point(32, 617, 1),
Point(223, 892, 1),
Point(28, 613, 1),
Point(225, 921, 1),
Point(40, 618, 1),
Point(234, 944, 1),
Point(42, 617, 1),
Point(222, 938, 1),
Point(34, 621, 1),
Point(195, 923, 1),
Point(55, 630, 1),
Point(471, 657, 1),
Point(348, 773, 1),
Point(493, 659, 1),
Point(314, 787, 1),
Point(509, 656, 1),
Point(311, 787, 1),
Point(481, 662, 1),
Point(310, 788, 1),
Point(486, 657, 1),
Point(306, 785, 1),
Point(494, 649, 1),
Point(307, 785, 1),
Point(506, 646, 1),
Point(303, 785, 1),
Point(495, 634, 1),
Point(298, 781, 1),
Point(463, 619, 1),
Point(300, 774, 1),
Point(458, 604, 1),
Point(298, 774, 1),
Point(452, 590, 1),
Point(294, 766, 1),
Point(443, 572, 1),
Point(288, 755, 1),
Point(434, 557, 1),
Point(279, 748, 1),
Point(425, 540, 1),
Point(278, 738, 1),
Point(417, 525, 1),
Point(273, 727, 1),
Point(409, 512, 1),
Point(269, 719, 1),
Point(402, 502, 1),
Point(265, 708, 1),
Point(396, 492, 1),
Point(261, 697, 1),
Point(389, 482, 1),
Point(257, 689, 1),
Point(384, 473, 1),
Point(255, 681, 1),
Point(377, 464, 1),
Point(249, 670, 1),
Point(371, 460, 1),
Point(245, 662, 1),
Point(366, 455, 1),
Point(241, 654, 1),
Point(359, 450, 1),
Point(236, 646, 1),
Point(354, 446, 1),
Point(233, 644, 1),
Point(349, 445, 1),
Point(230, 641, 1),
Point(342, 443, 1),
Point(224, 638, 1),
Point(336, 443, 1),
Point(224, 640, 1),
Point(332, 443, 1),
Point(215, 636, 1),
Point(327, 443, 1),
Point(211, 636, 1),
Point(324, 442, 1),
Point(204, 634, 1),
Point(322, 443, 1),
Point(201, 632, 1),
Point(322, 443, 1),
Point(202, 634, 1),
Point(321, 444, 1),
Point(201, 634, 1),
Point(321, 448, 1),
Point(203, 635, 1),
Point(325, 450, 1),
Point(201, 634, 1),
Point(332, 450, 1),
Point(206, 637, 1),
Point(345, 451, 1),
Point(207, 635, 1),
Point(362, 454, 1),
Point(214, 636, 1),
Point(380, 456, 1),
Point(219, 631, 1),
Point(403, 457, 1),
Point(233, 630, 1),
Point(433, 462, 1),
Point(245, 631, 1),
Point(455, 468, 1),
Point(257, 633, 1),
Point(480, 469, 1),
Point(273, 642, 1),
Point(498, 474, 1),
Point(290, 643, 1),
Point(506, 476, 1),
Point(300, 644, 1),
Point(522, 485, 1),
Point(311, 639, 1),
Point(527, 488, 1),
Point(320, 646, 1),
Point(532, 490, 1),
Point(325, 649, 1),
Point(534, 490, 1),
Point(327, 657, 1),
Point(531, 491, 1),
Point(328, 656, 1),
Point(532, 493, 1),
Point(326, 655, 1),
Point(526, 493, 1),
Point(322, 656, 1),
Point(518, 494, 1),
Point(314, 658, 1),
Point(507, 490, 1),
Point(306, 655, 1),
Point(506, 490, 1),
Point(299, 657, 1),
Point(496, 488, 1),
Point(291, 655, 1),
Point(484, 483, 1),
Point(283, 659, 1),
Point(476, 479, 1),
Point(276, 658, 1),
Point(462, 479, 1),
Point(269, 657, 1),
Point(455, 486, 1),
Point(263, 655, 1),
Point(442, 482, 1),
Point(255, 656, 1),
Point(429, 481, 1),
Point(244, 651, 1),
Point(417, 480, 1),
Point(237, 650, 1),
Point(403, 480, 1),
Point(226, 649, 1),
Point(389, 481, 1),
Point(214, 643, 1),
Point(377, 480, 1),
Point(205, 639, 1),
Point(365, 479, 1),
Point(199, 639, 1),
Point(353, 479, 1),
Point(196, 642, 1),
Point(339, 479, 1),
Point(197, 648, 1),
Point(329, 479, 1),
Point(204, 658, 1),
Point(326, 478, 1),
Point(202, 661, 1),
Point(322, 479, 1),
Point(193, 662, 1),
Point(314, 480, 1),
Point(193, 666, 1),
Point(306, 479, 1),
Point(193, 668, 1),
Point(305, 477, 1),
Point(191, 667, 1),
Point(306, 477, 1),
Point(192, 667, 1),
Point(307, 476, 1),
Point(192, 668, 1),
Point(311, 476, 1),
Point(194, 666, 1),
Point(315, 476, 1),
Point(197, 666, 1),
Point(324, 476, 1),
Point(197, 664, 1),
Point(330, 476, 1),
Point(198, 662, 1),
Point(348, 480, 1),
Point(196, 657, 1),
Point(366, 480, 1),
Point(203, 653, 1),
Point(384, 482, 1),
Point(211, 650, 1),
Point(406, 484, 1),
Point(227, 649, 1),
Point(434, 488, 1),
Point(238, 648, 1),
Point(457, 492, 1),
Point(251, 652, 1),
Point(460, 484, 1),
Point(264, 661, 1),
Point(494, 494, 1),
Point(278, 661, 1),
Point(490, 489, 1),
Point(289, 672, 1),
Point(507, 494, 1),
Point(295, 670, 1),
Point(501, 494, 1),
Point(297, 674, 1),
Point(506, 497, 1),
Point(297, 668, 1),
Point(505, 489, 1),
Point(299, 674, 1),
Point(503, 494, 1),
Point(296, 674, 1),
Point(493, 492, 1),
Point(293, 674, 1),
Point(484, 494, 1),
Point(285, 671, 1),
Point(475, 493, 1),
Point(278, 670, 1),
Point(468, 488, 1),
Point(272, 667, 1),
Point(459, 493, 1),
Point(262, 664, 1),
Point(451, 495, 1),
Point(250, 667, 1),
Point(434, 493, 1),
Point(242, 666, 1),
Point(419, 491, 1),
Point(237, 669, 1),
Point(397, 489, 1),
Point(223, 664, 1),
Point(380, 488, 1),
Point(216, 669, 1),
Point(365, 488, 1),
Point(214, 669, 1),
Point(347, 487, 1),
Point(203, 668, 1),
Point(334, 488, 1),
Point(197, 665, 1),
Point(316, 485, 1),
Point(195, 672, 1),
Point(308, 484, 1),
Point(192, 677, 1),
Point(295, 487, 1),
Point(184, 678, 1),
Point(286, 486, 1),
Point(180, 677, 1),
Point(284, 487, 1),
Point(180, 678, 1),
Point(281, 486, 1),
Point(176, 679, 1),
Point(277, 483, 1),
Point(179, 679, 1),
Point(275, 484, 1),
Point(178, 680, 1),
Point(276, 483, 1),
Point(178, 681, 1),
Point(278, 483, 1),
Point(178, 680, 1),
Point(278, 482, 1),
Point(180, 681, 1),
Point(280, 481, 1),
Point(181, 679, 1),
Point(288, 481, 1),
Point(182, 679, 1),
Point(295, 481, 1),
Point(183, 676, 1),
Point(308, 482, 1),
Point(188, 676, 1),
Point(320, 482, 1),
Point(187, 675, 1),
Point(339, 486, 1),
Point(191, 672, 1),
Point(362, 489, 1),
Point(192, 662, 1),
Point(378, 491, 1),
Point(207, 664, 1),
Point(403, 492, 1),
Point(219, 661, 1),
Point(424, 496, 1),
Point(232, 668, 1),
Point(444, 499, 1),
Point(244, 670, 1),
Point(461, 504, 1),
Point(255, 674, 1),
Point(476, 505, 1),
Point(261, 670, 1),
Point(476, 500, 1),
Point(267, 679, 1),
Point(483, 500, 1),
Point(277, 685, 1),
Point(486, 506, 1),
Point(278, 685, 1),
Point(488, 505, 1),
Point(280, 686, 1),
Point(490, 508, 1),
Point(276, 684, 1),
Point(483, 507, 1),
Point(273, 684, 1),
Point(478, 506, 1),
Point(269, 678, 1),
Point(471, 506, 1),
Point(260, 678, 1),
Point(456, 508, 1),
Point(256, 678, 1),
Point(445, 505, 1),
Point(248, 679, 1),
Point(430, 502, 1),
Point(235, 676, 1),
Point(413, 500, 1),
Point(229, 677, 1),
Point(397, 497, 1),
Point(218, 671, 1),
Point(384, 497, 1),
Point(209, 667, 1),
Point(369, 494, 1),
Point(205, 669, 1),
Point(357, 492, 1),
Point(198, 664, 1),
Point(346, 490, 1),
Point(195, 664, 1),
Point(336, 489, 1),
Point(194, 664, 1),
Point(328, 488, 1),
Point(196, 669, 1),
Point(322, 486, 1),
Point(192, 669, 1),
Point(320, 484, 1),
Point(192, 670, 1),
Point(322, 482, 1),
Point(194, 673, 1),
Point(323, 482, 1),
Point(196, 672, 1),
Point(326, 481, 1),
Point(188, 667, 1),
Point(335, 482, 1),
Point(191, 662, 1),
Point(348, 482, 1),
Point(201, 663, 1),
Point(360, 484, 1),
Point(204, 660, 1),
Point(376, 484, 1),
Point(216, 663, 1),
Point(395, 488, 1),
Point(228, 665, 1),
Point(420, 489, 1),
Point(237, 668, 1),
Point(443, 493, 1),
Point(251, 671, 1),
Point(485, 496, 1),
Point(264, 672, 1),
Point(485, 501, 1),
Point(278, 679, 1),
Point(500, 503, 1),
Point(292, 681, 1),
Point(507, 502, 1),
Point(307, 687, 1),
Point(517, 501, 1),
Point(312, 685, 1),
Point(527, 506, 1),
Point(319, 684, 1),
Point(532, 514, 1),
Point(326, 687, 1),
Point(535, 517, 1),
Point(329, 690, 1),
Point(536, 520, 1),
Point(331, 690, 1),
Point(534, 520, 1),
Point(330, 695, 1),
Point(532, 521, 1),
Point(327, 696, 1),
Point(522, 525, 1),
Point(324, 703, 1),
Point(523, 526, 1),
Point(322, 704, 1),
Point(510, 512, 1),
Point(322, 711, 1),
Point(466, 527, 1),
Point(316, 720, 1),
Point(425, 546, 1),
Point(312, 725, 1),
Point(392, 558, 1),
Point(307, 729, 1),
Point(358, 575, 1),
Point(305, 739, 1),
Point(328, 600, 1),
Point(303, 749, 1),
Point(308, 620, 1),
Point(301, 758, 1),
Point(294, 637, 1),
Point(300, 770, 1),
Point(279, 654, 1),
Point(295, 773, 1),
Point(267, 667, 1),
Point(296, 782, 1),
Point(256, 680, 1),
Point(294, 796, 1),
Point(251, 690, 1),
Point(295, 805, 1),
Point(243, 700, 1),
Point(291, 804, 1),
Point(241, 705, 1),
Point(292, 805, 1),
Point(238, 713, 1),
Point(283, 810, 1),
Point(235, 721, 1),
Point(292, 810, 1),
Point(233, 728, 1),
Point(296, 808, 1),
Point(231, 729, 1),
Point(296, 814, 1),
Point(230, 729, 1),
Point(295, 812, 1),
Point(234, 728, 1),
Point(296, 807, 1),
Point(233, 730, 1),
Point(299, 814, 1),
Point(233, 728, 1),
Point(293, 814, 1),
Point(234, 728, 1),
Point(290, 820, 1),
Point(234, 727, 1),
Point(285, 820, 1),
Point(233, 728, 1),
Point(290, 822, 1),
Point(234, 727, 1),
Point(282, 823, 1),
Point(232, 729, 1),
Point(285, 823, 1),
Point(231, 730, 1),
Point(284, 822, 1),
Point(232, 732, 1),
Point(289, 821, 1),
Point(230, 734, 1),
Point(286, 827, 1),
Point(230, 735, 1),
Point(282, 828, 1),
Point(230, 735, 1),
Point(288, 827, 1),
Point(231, 734, 1),
Point(289, 825, 1),
])
left = Template('left', [
Point(300, 296, 1),
Point(494, 383, 1),
Point(280, 296, 1),
Point(492, 382, 1),
Point(255, 294, 1),
Point(483, 378, 1),
Point(233, 292, 1),
Point(471, 378, 1),
Point(216, 288, 1),
Point(465, 382, 1),
Point(203, 287, 1),
Point(457, 378, 1),
Point(189, 281, 1),
Point(446, 379, 1),
Point(177, 278, 1),
Point(433, 377, 1),
Point(163, 272, 1),
Point(420, 377, 1),
Point(147, 272, 1),
Point(407, 376, 1),
Point(132, 272, 1),
Point(390, 375, 1),
Point(115, 275, 1),
Point(374, 373, 1),
Point(101, 278, 1),
Point(359, 373, 1),
Point(89, 280, 1),
Point(345, 370, 1),
Point(78, 281, 1),
Point(329, 367, 1),
Point(67, 286, 1),
Point(317, 367, 1),
Point(59, 289, 1),
Point(307, 368, 1),
Point(53, 291, 1),
Point(300, 369, 1),
Point(47, 294, 1),
Point(291, 370, 1),
Point(43, 295, 1),
Point(288, 374, 1),
Point(42, 297, 1),
Point(288, 372, 1),
Point(40, 298, 1),
Point(288, 379, 1),
Point(43, 300, 1),
Point(292, 377, 1),
Point(46, 300, 1),
Point(293, 377, 1),
Point(49, 302, 1),
Point(297, 376, 1),
Point(54, 304, 1),
Point(303, 381, 1),
Point(60, 304, 1),
Point(309, 379, 1),
Point(67, 305, 1),
Point(317, 381, 1),
Point(74, 306, 1),
Point(326, 383, 1),
Point(83, 306, 1),
Point(336, 386, 1),
Point(93, 307, 1),
Point(347, 389, 1),
Point(102, 308, 1),
Point(357, 390, 1),
Point(112, 310, 1),
Point(370, 393, 1),
Point(122, 314, 1),
Point(385, 397, 1),
Point(132, 317, 1),
Point(394, 398, 1),
Point(140, 320, 1),
Point(403, 399, 1),
Point(144, 321, 1),
Point(409, 401, 1),
Point(150, 323, 1),
Point(414, 402, 1),
Point(151, 322, 1),
Point(417, 404, 1),
Point(152, 322, 1),
Point(419, 404, 1),
Point(152, 323, 1),
Point(419, 404, 1),
Point(151, 324, 1),
Point(417, 404, 1),
Point(148, 324, 1),
Point(415, 403, 1),
Point(144, 325, 1),
Point(411, 402, 1),
Point(140, 326, 1),
Point(404, 401, 1),
Point(131, 327, 1),
Point(398, 400, 1),
Point(125, 327, 1),
Point(391, 399, 1),
Point(116, 326, 1),
Point(382, 399, 1),
Point(106, 326, 1),
Point(371, 398, 1),
Point(97, 323, 1),
Point(358, 395, 1),
Point(89, 321, 1),
Point(345, 393, 1),
Point(77, 317, 1),
Point(329, 389, 1),
Point(69, 316, 1),
Point(316, 387, 1),
Point(58, 315, 1),
Point(302, 388, 1),
Point(49, 313, 1),
Point(289, 389, 1),
Point(42, 312, 1),
Point(278, 385, 1),
Point(35, 312, 1),
Point(266, 385, 1),
Point(27, 313, 1),
Point(253, 382, 1),
Point(21, 312, 1),
Point(245, 379, 1),
Point(16, 311, 1),
Point(239, 380, 1),
Point(13, 312, 1),
Point(234, 379, 1),
Point(9, 313, 1),
Point(232, 381, 1),
Point(7, 314, 1),
Point(231, 383, 1),
Point(7, 315, 1),
Point(232, 385, 1),
Point(8, 316, 1),
Point(234, 385, 1),
Point(9, 316, 1),
Point(238, 385, 1),
Point(11, 316, 1),
Point(241, 388, 1),
Point(14, 316, 1),
Point(245, 390, 1),
Point(17, 317, 1),
Point(252, 392, 1),
Point(22, 318, 1),
Point(260, 394, 1),
Point(28, 318, 1),
Point(266, 395, 1),
Point(34, 319, 1),
Point(274, 397, 1),
Point(42, 320, 1),
Point(284, 399, 1),
Point(49, 319, 1),
Point(297, 401, 1),
Point(56, 320, 1),
Point(307, 401, 1),
Point(63, 322, 1),
Point(318, 403, 1),
Point(70, 323, 1),
Point(326, 404, 1),
Point(77, 323, 1),
Point(333, 405, 1),
Point(82, 325, 1),
Point(340, 407, 1),
Point(86, 326, 1),
Point(345, 407, 1),
Point(88, 325, 1),
Point(348, 407, 1),
Point(90, 325, 1),
Point(349, 405, 1),
Point(89, 325, 1),
Point(347, 405, 1),
Point(89, 324, 1),
Point(345, 405, 1),
Point(86, 323, 1),
Point(343, 404, 1),
Point(82, 323, 1),
Point(339, 404, 1),
Point(78, 322, 1),
Point(333, 401, 1),
Point(73, 320, 1),
Point(325, 400, 1),
Point(69, 319, 1),
Point(317, 398, 1),
Point(61, 316, 1),
Point(308, 396, 1),
Point(54, 316, 1),
Point(299, 395, 1),
Point(46, 314, 1),
Point(288, 393, 1),
Point(40, 313, 1),
Point(277, 389, 1),
Point(32, 313, 1),
Point(266, 387, 1),
Point(26, 312, 1),
Point(255, 385, 1),
Point(20, 311, 1),
Point(246, 383, 1),
Point(13, 311, 1),
Point(239, 380, 1),
Point(10, 312, 1),
Point(233, 380, 1),
Point(5, 314, 1),
Point(227, 379, 1),
Point(0, 315, 1),
Point(223, 382, 1),
Point(-7, 316, 1),
Point(217, 381, 1),
Point(-16, 314, 1),
Point(215, 384, 1),
Point(2, 314, 1),
Point(217, 388, 1),
Point(-18, 315, 1),
Point(214, 387, 1),
Point(-13, 316, 1),
Point(218, 391, 1),
Point(-14, 318, 1),
Point(220, 397, 1),
Point(-11, 319, 1),
Point(226, 401, 1),
Point(-16, 322, 1),
Point(231, 406, 1),
Point(-12, 326, 1),
Point(237, 413, 1),
Point(-8, 336, 1),
Point(245, 425, 1),
Point(-16, 350, 1),
Point(250, 428, 1),
Point(-12, 366, 1),
Point(257, 439, 1),
Point(-8, 381, 1),
Point(265, 446, 1),
Point(4, 396, 1),
Point(273, 454, 1),
Point(2, 413, 1),
Point(278, 462, 1),
Point(-4, 439, 1),
Point(283, 469, 1),
Point(-12, 462, 1),
Point(289, 478, 1),
Point(-14, 490, 1),
Point(299, 492, 1),
Point(0, 513, 1),
Point(310, 503, 1),
Point(12, 554, 1),
Point(323, 512, 1),
Point(57, 571, 1),
Point(326, 521, 1),
Point(14, 633, 1),
Point(325, 533, 1),
Point(22, 644, 1),
Point(322, 550, 1),
Point(12, 655, 1),
Point(333, 569, 1),
Point(52, 669, 1),
Point(343, 593, 1),
Point(77, 699, 1),
Point(358, 607, 1),
Point(88, 723, 1),
Point(347, 606, 1),
Point(88, 785, 1),
Point(336, 602, 1),
Point(95, 855, 1),
Point(340, 612, 1),
Point(107, 892, 1),
Point(336, 624, 1),
Point(101, 886, 1),
Point(358, 639, 1),
Point(131, 863, 1),
Point(336, 662, 1),
Point(152, 866, 1),
Point(252, 758, 1),
Point(331, 518, 1),
Point(500, 571, 1),
Point(315, 503, 1),
Point(486, 573, 1),
Point(298, 486, 1),
Point(483, 561, 1),
Point(282, 473, 1),
Point(474, 555, 1),
Point(263, 464, 1),
Point(464, 546, 1),
Point(243, 458, 1),
Point(445, 544, 1),
Point(219, 455, 1),
Point(420, 538, 1),
Point(195, 457, 1),
Point(392, 538, 1),
Point(170, 461, 1),
Point(367, 538, 1),
Point(156, 464, 1),
Point(343, 540, 1),
Point(146, 467, 1),
Point(328, 541, 1),
Point(142, 470, 1),
Point(321, 541, 1),
Point(141, 472, 1),
Point(317, 543, 1),
Point(143, 473, 1),
Point(320, 542, 1),
Point(148, 471, 1),
Point(328, 542, 1),
Point(158, 472, 1),
Point(342, 542, 1),
Point(171, 471, 1),
Point(358, 541, 1),
Point(184, 470, 1),
Point(374, 540, 1),
Point(201, 466, 1),
Point(391, 542, 1),
Point(216, 462, 1),
Point(409, 543, 1),
Point(229, 461, 1),
Point(424, 543, 1),
Point(242, 461, 1),
Point(437, 546, 1),
Point(249, 458, 1),
Point(443, 546, 1),
Point(251, 455, 1),
Point(445, 543, 1),
Point(245, 453, 1),
Point(441, 542, 1),
Point(236, 452, 1),
Point(432, 541, 1),
Point(223, 451, 1),
Point(419, 541, 1),
Point(207, 453, 1),
Point(403, 540, 1),
Point(185, 456, 1),
Point(385, 540, 1),
Point(167, 457, 1),
Point(362, 539, 1),
Point(142, 462, 1),
Point(337, 540, 1),
Point(129, 464, 1),
Point(312, 539, 1),
Point(111, 467, 1),
Point(286, 538, 1),
Point(94, 470, 1),
Point(260, 537, 1),
Point(81, 476, 1),
Point(237, 537, 1),
Point(71, 479, 1),
Point(220, 539, 1),
Point(64, 483, 1),
Point(207, 538, 1),
Point(60, 485, 1),
Point(198, 539, 1),
Point(58, 484, 1),
Point(194, 539, 1),
Point(58, 483, 1),
Point(193, 536, 1),
Point(58, 482, 1),
Point(193, 537, 1),
Point(59, 482, 1),
Point(197, 536, 1),
Point(62, 482, 1),
Point(199, 537, 1),
Point(64, 482, 1),
Point(206, 539, 1),
Point(69, 482, 1),
Point(214, 540, 1),
Point(74, 483, 1),
Point(226, 542, 1),
Point(81, 485, 1),
Point(237, 544, 1),
Point(89, 487, 1),
Point(250, 550, 1),
Point(97, 493, 1),
Point(265, 556, 1),
Point(107, 501, 1),
Point(282, 564, 1),
Point(118, 508, 1),
Point(294, 577, 1),
Point(132, 522, 1),
Point(311, 591, 1),
Point(144, 543, 1),
Point(329, 610, 1),
Point(157, 569, 1),
Point(350, 635, 1),
Point(168, 604, 1),
Point(367, 660, 1),
Point(184, 645, 1),
Point(389, 696, 1),
Point(200, 703, 1),
Point(412, 727, 1),
Point(345, 680, 1),
Point(456, 752, 1),
Point(344, 668, 1),
Point(450, 771, 1),
Point(347, 652, 1),
Point(446, 757, 1),
Point(349, 633, 1),
Point(449, 746, 1),
Point(357, 619, 1),
Point(442, 724, 1),
Point(362, 594, 1),
Point(452, 707, 1),
Point(401, 601, 1),
Point(455, 679, 1),
Point(413, 575, 1),
Point(459, 657, 1),
Point(365, 540, 1),
Point(474, 632, 1),
Point(367, 531, 1),
Point(493, 606, 1),
Point(361, 507, 1),
Point(502, 582, 1),
Point(368, 493, 1),
Point(518, 565, 1),
Point(371, 478, 1),
Point(528, 544, 1),
Point(371, 469, 1),
Point(532, 529, 1),
Point(370, 457, 1),
Point(529, 526, 1),
Point(369, 450, 1),
Point(531, 526, 1),
Point(365, 445, 1),
Point(525, 524, 1),
Point(362, 441, 1),
Point(530, 522, 1),
Point(353, 440, 1),
Point(530, 514, 1),
Point(342, 437, 1),
Point(518, 516, 1),
Point(338, 433, 1),
Point(507, 514, 1),
Point(333, 431, 1),
Point(501, 515, 1),
Point(324, 431, 1),
Point(494, 507, 1),
Point(311, 435, 1),
Point(483, 506, 1),
Point(300, 437, 1),
Point(476, 508, 1),
Point(287, 440, 1),
Point(472, 510, 1),
Point(273, 444, 1),
Point(464, 512, 1),
Point(264, 446, 1),
Point(458, 520, 1),
Point(253, 450, 1),
Point(450, 520, 1),
Point(240, 454, 1),
Point(439, 520, 1),
Point(231, 459, 1),
Point(430, 521, 1),
Point(223, 462, 1),
Point(426, 528, 1),
Point(216, 465, 1),
Point(422, 533, 1),
Point(210, 469, 1),
Point(416, 533, 1),
Point(206, 473, 1),
Point(414, 534, 1),
Point(204, 476, 1),
Point(412, 538, 1),
Point(204, 477, 1),
Point(415, 537, 1),
Point(208, 477, 1),
Point(417, 537, 1),
Point(211, 476, 1),
Point(422, 537, 1),
Point(217, 474, 1),
Point(426, 536, 1),
Point(223, 473, 1),
Point(431, 539, 1),
Point(230, 471, 1),
Point(440, 543, 1),
Point(238, 470, 1),
Point(447, 542, 1),
Point(247, 468, 1),
Point(453, 540, 1),
Point(254, 468, 1),
Point(459, 543, 1),
Point(261, 468, 1),
Point(464, 540, 1),
Point(272, 468, 1),
Point(465, 541, 1),
Point(281, 468, 1),
Point(472, 541, 1),
Point(289, 469, 1),
Point(477, 545, 1),
Point(297, 471, 1),
Point(483, 545, 1),
Point(305, 473, 1),
Point(486, 550, 1),
Point(312, 473, 1),
Point(490, 550, 1),
Point(317, 473, 1),
Point(495, 554, 1),
Point(322, 473, 1),
Point(502, 558, 1),
Point(324, 474, 1),
Point(508, 558, 1),
Point(325, 475, 1),
Point(508, 560, 1),
Point(321, 476, 1),
Point(504, 562, 1),
Point(316, 478, 1),
Point(500, 564, 1),
Point(309, 479, 1),
Point(489, 564, 1),
Point(302, 481, 1),
Point(484, 565, 1),
Point(292, 483, 1),
Point(477, 563, 1),
Point(282, 486, 1),
Point(471, 565, 1),
Point(268, 488, 1),
Point(463, 572, 1),
Point(257, 491, 1),
Point(457, 581, 1),
Point(244, 496, 1),
Point(447, 583, 1),
Point(228, 501, 1),
Point(435, 587, 1),
Point(217, 504, 1),
Point(423, 592, 1),
Point(203, 508, 1),
Point(416, 594, 1),
Point(190, 512, 1),
Point(406, 597, 1),
Point(181, 514, 1),
Point(399, 600, 1),
Point(173, 515, 1),
Point(394, 601, 1),
Point(171, 516, 1),
Point(390, 600, 1),
Point(167, 515, 1),
Point(387, 600, 1),
Point(167, 514, 1),
Point(387, 600, 1),
Point(170, 513, 1),
Point(388, 601, 1),
Point(175, 513, 1),
Point(392, 601, 1),
Point(183, 510, 1),
Point(399, 600, 1),
Point(191, 508, 1),
Point(406, 600, 1),
Point(199, 507, 1),
Point(414, 601, 1),
Point(208, 505, 1),
Point(423, 600, 1),
Point(219, 504, 1),
Point(432, 598, 1),
Point(231, 503, 1),
Point(443, 597, 1),
Point(242, 504, 1),
Point(448, 597, 1),
Point(255, 501, 1),
Point(459, 594, 1),
Point(265, 501, 1),
Point(467, 591, 1),
Point(274, 500, 1),
Point(471, 586, 1),
Point(285, 497, 1),
Point(474, 584, 1),
Point(292, 496, 1),
Point(476, 581, 1),
Point(299, 496, 1),
Point(480, 582, 1),
Point(304, 495, 1),
Point(484, 582, 1),
Point(305, 495, 1),
Point(487, 584, 1),
Point(305, 495, 1),
Point(487, 584, 1),
Point(302, 495, 1),
Point(486, 584, 1),
Point(298, 496, 1),
Point(481, 585, 1),
Point(293, 496, 1),
Point(480, 587, 1),
Point(285, 498, 1),
Point(478, 592, 1),
Point(275, 501, 1),
Point(474, 595, 1),
Point(266, 503, 1),
Point(467, 599, 1),
Point(252, 506, 1),
Point(457, 599, 1),
Point(238, 508, 1),
Point(443, 604, 1),
Point(225, 512, 1),
Point(435, 606, 1),
Point(210, 514, 1),
Point(424, 606, 1),
Point(198, 516, 1),
Point(413, 609, 1),
Point(185, 519, 1),
Point(406, 606, 1),
Point(173, 520, 1),
Point(395, 608, 1),
Point(166, 521, 1),
Point(388, 605, 1),
Point(159, 521, 1),
Point(382, 604, 1),
Point(154, 521, 1),
Point(378, 603, 1),
Point(154, 521, 1),
Point(375, 599, 1),
Point(154, 521, 1),
Point(375, 599, 1),
Point(158, 520, 1),
Point(377, 595, 1),
Point(161, 518, 1),
Point(379, 592, 1),
Point(169, 517, 1),
Point(385, 595, 1),
Point(177, 516, 1),
Point(396, 595, 1),
Point(184, 516, 1),
Point(404, 596, 1),
Point(193, 514, 1),
Point(414, 593, 1),
Point(205, 512, 1),
Point(425, 594, 1),
Point(216, 510, 1),
Point(436, 593, 1),
Point(228, 508, 1),
Point(445, 595, 1),
Point(240, 506, 1),
Point(449, 588, 1),
Point(249, 503, 1),
Point(457, 589, 1),
Point(258, 502, 1),
Point(464, 587, 1),
Point(267, 499, 1),
Point(471, 583, 1),
Point(274, 496, 1),
Point(476, 582, 1),
Point(280, 494, 1),
Point(478, 579, 1),
Point(283, 491, 1),
Point(478, 578, 1),
Point(286, 490, 1),
Point(479, 577, 1),
Point(285, 488, 1),
Point(480, 578, 1),
Point(282, 487, 1),
Point(477, 577, 1),
Point(277, 488, 1),
Point(473, 577, 1),
Point(270, 489, 1),
Point(470, 580, 1),
Point(261, 492, 1),
Point(462, 581, 1),
Point(251, 495, 1),
Point(458, 587, 1),
Point(241, 500, 1),
Point(449, 593, 1),
Point(224, 506, 1),
Point(438, 597, 1),
Point(213, 511, 1),
Point(428, 599, 1),
Point(200, 517, 1),
Point(416, 604, 1),
Point(187, 523, 1),
Point(407, 604, 1),
Point(177, 529, 1),
Point(397, 611, 1),
Point(164, 534, 1),
Point(387, 610, 1),
Point(155, 537, 1),
Point(380, 612, 1),
Point(150, 540, 1),
Point(373, 614, 1),
Point(144, 542, 1),
Point(368, 618, 1),
Point(139, 543, 1),
Point(363, 617, 1),
Point(135, 543, 1),
Point(359, 620, 1),
Point(134, 544, 1),
Point(358, 619, 1),
Point(135, 542, 1),
Point(359, 621, 1),
Point(136, 542, 1),
Point(358, 625, 1),
Point(140, 540, 1),
Point(361, 624, 1),
Point(144, 537, 1),
Point(366, 625, 1),
Point(149, 535, 1),
Point(371, 626, 1),
Point(154, 535, 1),
Point(377, 628, 1),
Point(159, 534, 1),
Point(381, 632, 1),
Point(165, 533, 1),
Point(386, 634, 1),
Point(171, 532, 1),
Point(390, 630, 1),
Point(176, 530, 1),
Point(395, 635, 1),
Point(183, 531, 1),
Point(399, 636, 1),
Point(188, 531, 1),
Point(404, 640, 1),
Point(192, 531, 1),
Point(408, 641, 1),
Point(197, 532, 1),
Point(411, 642, 1),
Point(202, 533, 1),
Point(415, 645, 1),
Point(208, 533, 1),
Point(418, 647, 1),
Point(214, 535, 1),
Point(422, 647, 1),
Point(217, 536, 1),
Point(426, 649, 1),
Point(225, 537, 1),
Point(430, 651, 1),
Point(230, 537, 1),
Point(437, 652, 1),
Point(235, 538, 1),
Point(439, 653, 1),
Point(243, 538, 1),
Point(444, 655, 1),
Point(248, 538, 1),
Point(451, 655, 1),
Point(255, 539, 1),
Point(454, 659, 1),
Point(259, 538, 1),
Point(459, 655, 1),
Point(265, 538, 1),
Point(462, 656, 1),
Point(268, 538, 1),
Point(465, 653, 1),
Point(275, 538, 1),
Point(468, 651, 1),
Point(279, 539, 1),
Point(473, 649, 1),
Point(284, 539, 1),
Point(475, 649, 1),
Point(289, 539, 1),
Point(478, 648, 1),
Point(292, 538, 1),
Point(481, 645, 1),
Point(296, 537, 1),
Point(484, 644, 1),
Point(299, 535, 1),
Point(485, 641, 1),
Point(301, 534, 1),
Point(485, 643, 1),
Point(305, 535, 1),
Point(487, 640, 1),
Point(308, 534, 1),
Point(487, 639, 1),
Point(310, 533, 1),
Point(489, 639, 1),
Point(311, 533, 1),
Point(490, 636, 1),
Point(314, 532, 1),
Point(490, 637, 1),
])
recognizer = Recognizer([right, left])

In [15]:
def run_servers(control_mode):
    """
    Starts server threads and keeps them running in a while loop.
    """
    threading.Thread(target=start_server, args=(3000, "Java")).start()
    threading.Thread(target=start_server, args=(4200, "Unity")).start()

    cap = cv2.VideoCapture(3)
    framecnt = 0

    frame = None

    last_position = None

    face_found = False
    frame_has_faces = False

    current_user_age = 18
    current_user_expression = "None"

    all_points = []
    classified_gesture = "None"
    results = None

    try:
        while True:
            # Read the frame from the camera
            ret, frame = cap.read()

            if not ret:
                print("Failed to read frame from camera.")
                break

            frame = cv2.resize(frame, (0, 0), fx=0.50, fy=0.50)
            framecnt += 1

            # Gesture Control
            if control_mode == 1:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = hands.process(rgb_frame)

                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks:
                        image_height, image_width, _ = frame.shape
                        x_tip = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width)
                        y_tip = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height)
                        all_points.append(Point(x_tip, y_tip, 1))

                        x_wrist = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * image_width)
                        y_wrist = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * image_height)
                        all_points.append(Point(x_wrist, y_wrist, 1))

            if framecnt % 23 == 0:
                # LED Tracking
                if control_mode == 2:
                    led_tracked_frame, current_position = detect_led(frame, lower_red, upper_red)

                    if current_position is not None:
                        if last_position is not None:
                            # Calculate the position difference
                            dx = current_position[0] - last_position[0]
                            dy = current_position[1] - last_position[1]
                            send_structured_message("Unity", "RotateModel", dx)

                        # Update the last position
                        last_position = current_position

                # Facial Recognition
                face_locations, face_names, frame_has_faces = detect_faces(frame, known_face_encodings, known_face_names)
                new_face_found = len(face_names) > 0
                if new_face_found != face_found:
                    face_found = new_face_found
                    send_structured_message("Java", "FaceIdentification", face_found)

                # Gaze Tracking
                gaze_detection_frame = track_gaze(frame)

                # Gesture Recognition
                if len(all_points) > 0 and control_mode == 1:
                    result = recognizer.recognize(all_points)
                    classified_gesture = result[0]
                    all_points.clear()

                    if classified_gesture in ['right', 'left']:
                        send_structured_message("Unity", "RotateModel", 1 if classified_gesture == 'right' else -1)

            if framecnt % 69 == 0 and frame_has_faces:
                framecnt = 0

                # Emotion + Age detection
                required_outputs =  ['age', 'emotion']
                result = DeepFace.analyze(frame, actions = required_outputs, enforce_detection = False)
                if len(result) > 0:
                    current_user_expression = result[0]["dominant_emotion"]

                    if result[0]["age"] != current_user_age:
                        current_user_age = result[0]["age"]
                        send_structured_message("Java", "UserAge", True if current_user_age >= 16 else False)

            cv2.putText(frame, f"Expression: {current_user_expression}", (5, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Age: {current_user_age}", (5, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

            if control_mode == 1:
                cv2.putText(frame, f"Gesture: {classified_gesture}", (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                if results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Display the frame
            cv2.imshow('Python Server', frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except KeyboardInterrupt:
        print("Shutting down servers...")
    finally:
        close_all_connections()
        cap.release()
        cv2.destroyAllWindows()
        
        df = save_gaze_data_to_csv()

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_name = f"./gazedata/gaze_data_{timestamp}.png"
        frame_shape = (frame.shape[1], frame.shape[0])
        generate_heatmap(df, frame_shape, file_name)

In [ ]:
control_mode = int(input("Choose control method: Enter '1' for Gesture Control or '2' for LED Control: "))
while control_mode not in [1, 2]:
    control_mode = int(input("Invalid control mode!"))

run_servers(control_mode)

Choose control method: Enter '1' for Gesture Control or '2' for LED Control:  2


Server listening for Java on port 3000...
Server listening for Unity on port 4200...
Java is not connected. {"Action": "FaceIdentification", "Value": true}
 can't be sent.
Java is not connected. {"Action": "FaceIdentification", "Value": false}
 can't be sent.
Java is not connected. {"Action": "FaceIdentification", "Value": true}
 can't be sent.


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.32it/s]


Java is not connected. {"Action": "UserAge", "Value": true}
 can't be sent.


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.99it/s]


...

Sending message: b'{"Action": "UserAge", "Value": true}\n'
Sending message: b'{"Action": "FaceIdentification", "Value": false}\n'
Sending message: b'{"Action": "FaceIdentification", "Value": true}\n'


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.53it/s]


Sending message: b'{"Action": "UserAge", "Value": true}\n'
Sending message: b'{"Action": "FaceIdentification", "Value": false}\n'
Sending message: b'{"Action": "FaceIdentification", "Value": true}\n'
Sending message: b'{"Action": "FaceIdentification", "Value": false}\n'
Sending message: b'{"Action": "FaceIdentification", "Value": true}\n'


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.71it/s]


Sending message: b'{"Action": "UserAge", "Value": true}\n'
Sending message: b'{"Action": "FaceIdentification", "Value": false}\n'
Sending message: b'{"Action": "FaceIdentification", "Value": true}\n'
Sending message: b'{"Action": "FaceIdentification", "Value": false}\n'
